# Indexación jerárquica

Hasta ahora nos hemos centrado principalmente en datos unidimensionales y bidimensionales, almacenados en objetos Pandas ``Series`` y ``DataFrame``, respectivamente.
A menudo es útil ir más allá y almacenar datos de mayor dimensión, es decir, datos indexados por más de una o dos claves.
Aunque Pandas proporciona objetos ``Panel`` y ``Panel4D`` que manejan de forma nativa datos tridimensionales y cuatridimensionales (ver [Aside: Panel Data](#Aside:-Panel-Data)), un patrón mucho más común en la práctica es hacer uso de la *indexación jerárquica* (también conocida como *multi-indexación*) para incorporar múltiples *niveles de índice* dentro de un único índice.
De este modo, los datos de mayor dimensión pueden representarse de forma compacta dentro de los conocidos objetos unidimensionales ``Series`` y bidimensionales ``DataFrame``.


En esta sección, exploraremos la creación directa de los objetos ``MultiIndex``, las consideraciones a tener en cuenta a la hora de indexar, trocear y calcular las estadísticas de los datos con índices múltiples, y las rutinas útiles para convertir entre las representaciones simples y jerárquicas de los datos.

Comenzamos con las importaciones estándar:

In [ ]:
import pandas as pd
import numpy as np

## Una Serie Multiplicada Indexada

Empecemos por considerar cómo podríamos representar datos bidimensionales dentro de una ``Serie`` unidimensional.
Para concretar, consideraremos una serie de datos donde cada punto tiene un carácter y una clave numérica.

### La mala manera

Supongamos que quieres rastrear datos sobre estados de dos años diferentes.
Usando las herramientas de Pandas que ya hemos cubierto, podrías estar tentado a usar simplemente tuplas de Python como claves:

In [ ]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

Con este esquema de indexación, se puede indexar o trocear directamente la serie en función de este índice múltiple:

In [ ]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

Pero la comodidad termina ahí. Por ejemplo, si necesitas seleccionar todos los valores de 2010, tendrás que hacer un poco de munching desordenado (y potencialmente lento) para lograrlo:

In [ ]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

Esto produce el resultado deseado, pero no es tan limpio (o tan eficiente para grandes conjuntos de datos) como la sintaxis de corte que tanto nos gusta en Pandas.

### The Better Way: Pandas MultiIndex
Afortunadamente, Pandas proporciona una forma mejor.
Nuestra indexación basada en tuplas es esencialmente un multiíndice rudimentario, y el tipo Pandas ``MultiIndex`` nos da el tipo de operaciones que deseamos tener.
Podemos crear un multiíndice a partir de las tuplas de la siguiente manera:

In [ ]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

Observe que el ``MultiIndex`` contiene múltiples *niveles* de indexación -en este caso, los nombres de los estados y los años-, así como múltiples *etiquetas* para cada punto de datos que codifican estos niveles.

Si volvemos a indexar nuestra serie con este ``MultiIndex``, veremos la representación jerárquica de los datos:

In [ ]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Aquí las dos primeras columnas de la representación ``Series`` muestran los valores de los índices múltiples, mientras que la tercera columna muestra los datos.
Obsérvese que faltan algunas entradas en la primera columna: en esta representación de índices múltiples, cualquier entrada en blanco indica el mismo valor que la línea superior.

Ahora, para acceder a todos los datos para los que el segundo índice es 2010, podemos simplemente utilizar la notación de corte de Pandas:

In [ ]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

El resultado es un array indexado individualmente con sólo las claves que nos interesan.
Esta sintaxis es mucho más conveniente (¡y la operación es mucho más eficiente!) que la solución de indexación múltiple basada en tuplas con la que empezamos.
A continuación, discutiremos este tipo de operación de indexación en datos indexados jerárquicamente.

### MultiIndex como dimensión extra

Puedes notar algo más aquí: podríamos fácilmente haber almacenado los mismos datos usando un simple ``DataFrame`` con etiquetas de índice y columna.
De hecho, Pandas está construido con esta equivalencia en mente. El método ``unstack()`` convertirá rápidamente una ``Serie`` con índices múltiples en un ``DataFrame`` con índices convencionales:

In [ ]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Naturalmente, el método ``stack()`` proporciona la operación contraria:

In [ ]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Al ver esto, te preguntarás por qué nos molestamos en utilizar la indexación jerárquica.
La razón es sencilla: igual que hemos podido utilizar la multiindexación para representar datos bidimensionales dentro de una ``Serie`` unidimensional, también podemos utilizarla para representar datos de tres o más dimensiones en una ``Serie`` o ``DataFrame``.
Cada nivel extra en un multiíndice representa una dimensión extra de datos; aprovechar esta propiedad nos da mucha más flexibilidad en los tipos de datos que podemos representar. Concretamente, podríamos querer añadir otra columna de datos demográficos para cada estado en cada año (digamos, población menor de 18 años); con un ``MultiIndex`` esto es tan fácil como añadir otra columna al ``DataFrame``:

In [ ]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

Además, todas las ufuncs y otras funcionalidades discutidas en [Operando con datos en Pandas](03.03-Operaciones-en-Pandas.ipynb) funcionan también con índices jerárquicos.
Aquí calculamos la fracción de personas menores de 18 años por año, dados los datos anteriores:

In [ ]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


Esto nos permite manipular y explorar fácil y rápidamente incluso los datos de alta dimensión.

## Métodos de creación de índices múltiples

La forma más directa de construir una ``Serie`` o un ``DataFrame`` con índices múltiples es simplemente pasar una lista de dos o más matrices de índices al constructor. Por ejemplo:

In [ ]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.554233  0.356072
  2  0.925244  0.219474
b 1  0.441759  0.610054
  2  0.171495  0.886688

El trabajo de crear el ``MultiIndex`` se hace en segundo plano.

Del mismo modo, si se pasa un diccionario con tuplas apropiadas como claves, Pandas lo reconocerá automáticamente y utilizará un ``MultiIndex`` por defecto:

In [ ]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Sin embargo, a veces es útil crear explícitamente un ``MultiIndex``; aquí veremos un par de estos métodos.

### Constructores explícitos de MultiIndex

Para una mayor flexibilidad en la construcción del índice, puedes utilizar los métodos constructores de la clase ``pd.MultiIndex``.
Por ejemplo, como hemos hecho antes, puedes construir el ``MultiIndex`` a partir de una simple lista de arrays que den los valores del índice dentro de cada nivel:

In [ ]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

Se puede construir a partir de una lista de tuplas que dan los valores de los índices múltiples de cada punto:

In [ ]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

Incluso se puede construir a partir de un producto cartesiano de índices simples:

In [ ]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

Del mismo modo, puedes construir el ``MultiIndex`` directamente utilizando su codificación interna pasando ``niveles`` (una lista de listas que contienen los valores de índice disponibles para cada nivel) y ``etiquetas`` (una lista de listas que hacen referencia a estas etiquetas):

In [ ]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

Cualquiera de estos objetos puede pasarse como argumento ``index`` cuando se crea una ``Serie`` o ``Dataframe``, o pasarse al método ``reindex`` de una ``Serie`` o ``DataFrame`` existente.

### Nombres de los niveles de MultiIndex

A veces es conveniente nombrar los niveles del ``MultiIndex``.
Esto se puede conseguir pasando el argumento ``names`` a cualquiera de los constructores de ``MultiIndex``, o estableciendo el atributo ``names`` del índice a posteriori:

In [ ]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Con conjuntos de datos más complicados, esto puede ser una forma útil de seguir el significado de varios valores de índice.

### MultiIndex para las columnas

En un ``DataFrame``, las filas y columnas son completamente simétricas, y al igual que las filas pueden tener múltiples niveles de índices, las columnas también pueden tener múltiples niveles.
Considere lo siguiente, que es una maqueta de algunos datos médicos (algo realistas):

In [ ]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      31.0  38.7  32.0  36.7  35.0  37.2
     2      44.0  37.7  50.0  35.0  29.0  36.7
2014 1      30.0  37.4  39.0  37.8  61.0  36.9
     2      47.0  37.8  48.0  37.3  51.0  36.5

Aquí vemos que la multiindexación de filas y columnas puede ser *muy* útil.
Se trata de datos fundamentalmente cuatridimensionales, donde las dimensiones son el sujeto, el tipo de medición, el año y el número de visita.
Con esto en su lugar podemos, por ejemplo, indexar la columna de nivel superior por el nombre de la persona y obtener un ``DataFrame`` completo que contenga sólo la información de esa persona:

In [ ]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      32.0  36.7
     2      50.0  35.0
2014 1      39.0  37.8
     2      48.0  37.3

Para los registros complicados que contienen múltiples mediciones etiquetadas a lo largo de múltiples tiempos para muchos sujetos (personas, países, ciudades, etc.) el uso de filas y columnas jerárquicas puede ser extremadamente conveniente.

## Indexación y corte de un MultiIndex

La indexación y el corte en un ``MultiIndex`` está diseñado para ser intuitivo, y ayuda si piensas en los índices como dimensiones añadidas.
Primero veremos la indexación de ``Series`` con índices múltiples, y luego la de ``DataFrame`` con índices múltiples.

### Series de índice múltiple

Considere la ``Serie`` de poblaciones de estados multi-indexada que vimos anteriormente:

In [ ]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Podemos acceder a elementos individuales mediante la indexación con términos múltiples:

In [ ]:
pop['California', 2000]

33871648

El ``MultiIndex`` también admite la *indexación parcial*, es decir, la indexación de sólo uno de los niveles del índice.
El resultado es otra ``Serie``, con los índices de nivel inferior mantenidos:

In [ ]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

El corte parcial también está disponible, siempre y cuando el ``MultiIndex`` esté ordenado (véase la discusión en [Índices ordenados y no ordenados](#Índices-ordenados-y-no-ordenados)):

In [ ]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

Con los índices ordenados, se puede realizar una indexación parcial en los niveles inferiores pasando un trozo vacío en el primer índice:

In [ ]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

Otros tipos de indexación y selección (discutidos en [Indexación y selección de datos](03.02-Indexación-y-selección-de-datos.ipynb)) también funcionan; por ejemplo, la selección basada en máscaras booleanas:

In [ ]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

También funciona la selección basada en la indexación de fantasía:

In [ ]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### DataFrames de índice múltiple

Un ``DataFrame`` de índice múltiple se comporta de manera similar.
Consideremos nuestro ``DataFrame`` médico de juguete de antes:

In [ ]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      31.0  38.7  32.0  36.7  35.0  37.2
     2      44.0  37.7  50.0  35.0  29.0  36.7
2014 1      30.0  37.4  39.0  37.8  61.0  36.9
     2      47.0  37.8  48.0  37.3  51.0  36.5

Recuerda que las columnas son primarias en un ``DataFrame``, y que la sintaxis utilizada para las ``Series`` con índices múltiples se aplica a las columnas.
Por ejemplo, podemos recuperar los datos de la frecuencia cardíaca de Guido con una simple operación:

In [ ]:
health_data['Guido', 'HR']

year  visit
2013  1        32.0
      2        50.0
2014  1        39.0
      2        48.0
Name: (Guido, HR), dtype: float64

También, como en el caso de un solo índice, podemos utilizar los indexadores ``loc``, ``iloc`` y ``ix`` introducidos en [Indexación y Selección de Datos](03.02-Indexación y Selección de Datos.ipynb). Por ejemplo:

In [ ]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      31.0  38.7
     2      44.0  37.7

Estos indexadores proporcionan una vista similar a un array de los datos bidimensionales subyacentes, pero a cada índice individual en ``loc`` o ``iloc`` se le puede pasar una tupla de múltiples índices. Por ejemplo:

In [ ]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        31.0
      2        44.0
2014  1        30.0
      2        47.0
Name: (Bob, HR), dtype: float64

Trabajar con rodajas dentro de estas tuplas de índice no es especialmente conveniente; intentar crear una rodaja dentro de una tupla conducirá a un error de sintaxis:

In [ ]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-32-8e3cc151e316>, line 1)

Se podría evitar esto construyendo la porción deseada explícitamente usando la función incorporada de Python ``slice()``, pero una mejor manera en este contexto es usar un objeto ``IndexSlice``, que Pandas proporciona precisamente para esta situación.
Por ejemplo:

In [ ]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,31.0,32.0,35.0
2014,1,30.0,39.0,61.0


Hay muchas formas de interactuar con los datos en las ``Series`` y los ``DataFrame`` multi-indexados, y al igual que con muchas de las herramientas de este libro, la mejor manera de familiarizarse con ellas es probarlas.

## Reordenación de los índices múltiples

Una de las claves para trabajar con datos de índices múltiples es saber cómo transformar los datos de forma efectiva.
Hay una serie de operaciones que preservarán toda la información del conjunto de datos, pero la reordenarán para los propósitos de varios cálculos.
Vimos un breve ejemplo de esto en los métodos ``stack()`` y ``unstack()``, pero hay muchas más formas de controlar finamente el reordenamiento de los datos entre índices jerárquicos y columnas, y las exploraremos aquí.

### Índices ordenados y no ordenados

Anteriormente, hemos mencionado brevemente una advertencia, pero deberíamos enfatizarla más aquí.
*Muchas de las operaciones de corte de ``MultiIndex`` fallarán si el índice no está ordenado.
Echemos un vistazo a esto.

Empezaremos creando algunos datos simples de índice múltiple donde los índices no están *ordenados lexográficamente*:

In [ ]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.003001
      2      0.164974
c     1      0.741650
      2      0.569264
b     1      0.001693
      2      0.526226
dtype: float64

Si intentamos tomar una porción parcial de este índice, se producirá un error:

In [ ]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'KeyError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


Aunque no está del todo claro en el mensaje de error, esto es el resultado de que el MultiIndex no esté ordenado.
Por varias razones, los cortes parciales y otras operaciones similares requieren que los niveles del ``MultiIndex`` estén ordenados (es decir, lexográficos).
Pandas proporciona un número de rutinas de conveniencia para realizar este tipo de ordenación; ejemplos son los métodos ``sort_index()`` y ``sortlevel()`` del ``DataFrame``.
Aquí utilizaremos el más sencillo, ``sort_index()``:

In [ ]:
data = data.sort_index()
data

char  int
a     1      0.003001
      2      0.164974
b     1      0.001693
      2      0.526226
c     1      0.741650
      2      0.569264
dtype: float64

Con el índice ordenado de esta manera, el corte parcial funcionará como se espera:

In [ ]:
data['a':'b']

char  int
a     1      0.003001
      2      0.164974
b     1      0.001693
      2      0.526226
dtype: float64

### Apilar y desapilar índices

Como hemos visto brevemente antes, es posible convertir un conjunto de datos de un índice múltiple apilado a una representación bidimensional simple, especificando opcionalmente el nivel a utilizar:

In [ ]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [ ]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Lo contrario de ``unstack()`` es ``stack()``, que aquí se puede utilizar para recuperar la serie original:

In [ ]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Ajuste y restablecimiento del índice

Otra forma de reorganizar los datos jerárquicos es convertir las etiquetas de los índices en columnas; esto se puede lograr con el método ``reset_index``.
Si se llama a este método en el diccionario de población, se obtendrá un ``DataFrame`` con una columna *estado* y *año* que contendrá la información que antes estaba en el índice.
Para mayor claridad, podemos especificar opcionalmente el nombre de los datos para la representación de la columna:

In [ ]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


A menudo, cuando se trabaja con datos en el mundo real, los datos de entrada en bruto tienen este aspecto y es útil construir un ``MultiIndex`` a partir de los valores de las columnas.
Esto se puede hacer con el método ``set_index`` del ``DataFrame``, que devuelve un ``DataFrame`` con índice múltiple:

In [ ]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

En la práctica, encuentro que este tipo de reindexación es uno de los patrones más útiles cuando me encuentro con conjuntos de datos del mundo real.

## Agregaciones de datos en multiíndices

Ya hemos visto que Pandas tiene métodos de agregación de datos incorporados, como ``media()``, ``suma()``, y ``máx()``.
Para los datos indexados jerárquicamente, se les puede pasar un parámetro de ``nivel`` que controla el subconjunto de datos sobre el que se calcula el agregado.

Por ejemplo, volvamos a nuestros datos de salud:

In [ ]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      31.0  38.7  32.0  36.7  35.0  37.2
     2      44.0  37.7  50.0  35.0  29.0  36.7
2014 1      30.0  37.4  39.0  37.8  61.0  36.9
     2      47.0  37.8  48.0  37.3  51.0  36.5

Quizás queramos promediar las mediciones en las dos visitas de cada año. Podemos hacerlo nombrando el nivel del índice que nos gustaría explorar, en este caso el año:

In [ ]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob       Guido          Sue       
type       HR  Temp    HR   Temp    HR   Temp
year                                         
2013     37.5  38.2  41.0  35.85  32.0  36.95
2014     38.5  37.6  43.5  37.55  56.0  36.70

Haciendo uso además de la palabra clave ``axis``, podemos tomar también la media entre los niveles de las columnas:

In [ ]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,36.833333,37.000000
2014,46.000000,37.283333


Así, en dos líneas, hemos podido encontrar la media de la frecuencia cardíaca y la temperatura medida entre todos los sujetos en todas las visitas de cada año.
Esta sintaxis es en realidad un atajo a la funcionalidad ``GroupBy``, de la que hablaremos en [Agregación y agrupación](03.08-Agregación-y-agrupación.ipynb).
Aunque este es un ejemplo de juguete, muchos conjuntos de datos del mundo real tienen una estructura jerárquica similar.

## Aside: Datos del panel

Pandas tiene otras estructuras de datos fundamentales que aún no hemos discutido, a saber, los objetos ``pd.Panel`` y ``pd.Panel4D``.
Estos pueden ser considerados, respectivamente, como generalizaciones tridimensionales y cuatridimensionales de las estructuras (unidimensionales) ``Series`` y (bidimensionales) ``DataFrame``.
Una vez que te hayas familiarizado con la indexación y manipulación de datos en una ``Serie`` y un ``DataFrame``, el ``Panel`` y el ``Panel4D`` son relativamente sencillos de utilizar.
En particular, los indexadores ``ix``, ``loc`` y ``iloc`` discutidos en [Indexación y selección de datos](03.02-DIndexación-y-selección-de-datos.ipynb) se extienden fácilmente a estas estructuras de mayor dimensión.

No cubriremos más estas estructuras de panel en este texto, ya que he encontrado en la mayoría de los casos que la multi-indexación es una representación más útil y conceptualmente más simple para los datos de mayor dimensión.
Además, los datos de panel son fundamentalmente una representación de datos densos, mientras que la multiindexación es fundamentalmente una representación de datos dispersos.
A medida que aumenta el número de dimensiones, la representación densa puede resultar muy ineficiente para la mayoría de los conjuntos de datos del mundo real.
Sin embargo, para las aplicaciones especializadas ocasionales, estas estructuras pueden ser útiles.
Si desea leer más sobre las estructuras ``Panel`` y ``Panel4D``, consulte las referencias que aparecen en [Recursos adicionales](03.13-Recursos adicionales.ipynb).